In [2]:
from datasets import load_dataset

In [4]:
raw_dataset = load_dataset("jmamou/augmented-glue-sst2", "mrpc")
raw_dataset

Downloading:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Using custom data configuration mrpc


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/107M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset augmented_glue_sst2 downloaded and prepared to C:\Users\aonno\.cache\huggingface\datasets\jmamou___augmented_glue_sst2\mrpc\0.0.0\247b2b4c3f1b4f6143d2de4b57cb43c1464ff5a3b0b46eeac207ca6795138475. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'prediction'],
        num_rows: 797056
    })
})

In [5]:
raw_train_dataset = raw_dataset["train"]
raw_train_dataset[0]

{'sentence': "tways smartly through heroines' transcendent stages of fear and intrigue and away from tragic characters with uncanny and hilarious humor .",
 'label': 1,
 'prediction': [0.00044510114821605384, 0.9995549321174622]}

In [6]:
raw_train_dataset.features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['0', '1'], names_file=None, id=None),
 'prediction': Sequence(feature=Value(dtype='float32', id=None), length=2, id=None)}

In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_dataset["train"]["sentence"])
tokenized_sentences_2 = tokenizer(raw_dataset["train"]["sentence"])

In [12]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [14]:
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/798 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'prediction', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 797056
    })
})

In [21]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence"]}
[len(x) for x in samples["input_ids"]]

[30, 23, 44, 13, 16, 19, 21, 39]

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'prediction': torch.Size([8, 2]),
 'input_ids': torch.Size([8, 44]),
 'token_type_ids': torch.Size([8, 44]),
 'attention_mask': torch.Size([8, 44]),
 'labels': torch.Size([8])}